In [1]:
# Bibliotecas necessárias
import pygame               # Biblioteca para criação de jogos e gráficos 2D e 3D. 
from pygame.locals import * # Importa todas as constantes e funções locais do módulo pygame.locals
from OpenGL.GL import *     # Importa todas as funções necessárias para o uso do OpenGL (para construção e manipulação de gráficos 3D)
from OpenGL.GLU import *    # Importa funções adicionais para OpenGL que ajudam na configuração de câmeras e projeções
from PIL import Image       # Biblioteca para manipulação de imagens, utilizada para carregar, editar e processar texturas

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Função para carregar textura
def load_texture(filename):
    texture = glGenTextures(1) # Gera um identificador único para a textura
    glBindTexture(GL_TEXTURE_2D, texture) # Vincula a textura ao alvo GL_TEXTURE_2D para realizar operações sobre ela
    image = Image.open(filename) # Carrega a imagem usando a biblioteca Pillow
    image = image.transpose(Image.FLIP_TOP_BOTTOM) # Espelha a imagem verticalmente, pois o OpenGL lê a textura de baixo para cima
    img_data = image.convert("RGBA").tobytes() # Converte a imagem para o formato RGBA (se ainda não estiver) e extrai os bytes
    # Define a imagem como a textura atual no OpenGL
    # - GL_TEXTURE_2D: Tipo da textura (bidimensional)
    # - 0: Nível de detalhe (0 é o nível base)
    # - GL_RGBA: Formato interno da textura no OpenGL
    # - image.width, image.height: Largura e altura da imagem
    # - 0: Borda da textura (deve ser 0)
    # - GL_RGBA: Formato dos dados de imagem
    # - GL_UNSIGNED_BYTE: Tipo dos componentes dos pixels (valores entre 0 e 255)
    # - img_data: Dados da imagem em formato de bytes
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, image.width, image.height, 0, GL_RGBA, GL_UNSIGNED_BYTE, img_data)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT) # Configura como a textura deve se comportar no eixo horizontal (S) quando repetida
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT) # Configura como a textura deve se comportar no eixo vertical (T) quando repetida
    # - GL_LINEAR: Interpolação linear para suavizar a textura
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR) # Define o filtro para ampliação da textura (quando ampliada na tela)
    glTexParameterf(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR) # Define o filtro para redução da textura (quando reduzida na tela)
    return texture # Retorna o identificador único da textura, que pode ser usado em futuras renderizações

In [3]:
# Função para inicializar a janela
def init_window():
    pygame.init()  # Inicializa o módulo do Pygame, necessário para usar as funções
    display = (800, 600)  # Define as dimensões da janela (largura = 800, altura = 600)
    pygame.display.set_mode(display, DOUBLEBUF | OPENGL) # Cria a janela Pygame e a inicializa com um contexto OpenGL
    gluPerspective(45, (display[0] / display[1]), 0.1, 50.0) #Configura a projeção de perspectiva para o contexto OpenGL
    glTranslatef(0.0, -5.0, -20)  # Ajuste da posição da câmera para vista superior

In [4]:
# Função para configurar a iluminação
def init_lighting():
    glEnable(GL_LIGHTING)  # Ativa o sistema de iluminação no OpenGL
    glEnable(GL_LIGHT0)  # Ativa a luz número 0 (GL_LIGHT0) no sistema de iluminação
    glEnable(GL_DEPTH_TEST) # Habilita o teste de profundidade para garantir que os objetos sejam desenhados na ordem correta com base na distância
    glEnable(GL_COLOR_MATERIAL)  # Permite que o material dos objetos interaja com a luz, usando suas cores para cálculo de iluminação

    # Define a posição da luz GL_LIGHT0
    # Parâmetros: (x, y, z, w)
    # - w = 1 indica que a luz é uma fonte de luz posicional
    # - w = 0 faria da luz uma luz direcional (paralela, como a luz do sol)
    glLightfv(GL_LIGHT0, GL_POSITION, (-5, 10, 0, 1))  # Luz posicionada em (-5, 10, 0)

    # Define a cor e a intensidade da luz difusa para GL_LIGHT0
    # - A luz difusa simula como a luz é espalhada em várias direções ao atingir uma superfície
    # - Valores (1.0, 1.0, 1.0, 1.0) representam uma luz branca de intensidade máxima
    glLightfv(GL_LIGHT0, GL_DIFFUSE, (1.0, 1.0, 1.0, 1.0))

    # Define a cor e a intensidade da luz especular para GL_LIGHT0
    # - A luz especular cria reflexos brilhantes (pontos de destaque) em superfícies
    # - Valores (1.0, 1.0, 1.0, 1.0) representam reflexos brancos de intensidade máxima
    glLightfv(GL_LIGHT0, GL_SPECULAR, (1.0, 1.0, 1.0, 1.0))

    # Define a cor e a intensidade da luz ambiente para GL_LIGHT0
    # - A luz ambiente é uma iluminação uniforme que afeta todas as superfícies igualmente
    # - Valores (0.3, 0.3, 0.3, 1.0) representam uma luz ambiente fraca de cor cinza
    glLightfv(GL_LIGHT0, GL_AMBIENT, (0.3, 0.3, 0.3, 1.0))

In [5]:
# Função para desenhar um cubo texturizado (para o prédio)
def draw_textured_cube(texture):
    glBindTexture(GL_TEXTURE_2D, texture)  # Associa a textura fornecida ao objeto 3D.
    glBegin(GL_QUADS)  # Inicia a definição de quadrados (faces do cubo).
    # Define os vértices e coordenadas de textura para cada face do cubo.
    vertices = [
        [(1, -1, -1), (0, 0)], [(1, 1, -1), (1, 0)], [(1, 1, 1), (1, 1)], [(1, -1, 1), (0, 1)],      # Frente
        [(-1, -1, 1), (0, 0)], [(-1, 1, 1), (1, 0)], [(-1, 1, -1), (1, 1)], [(-1, -1, -1), (0, 1)],  # Trás
        [(-1, 1, -1), (0, 0)], [(-1, 1, 1), (1, 0)], [(1, 1, 1), (1, 1)], [(1, 1, -1), (0, 1)],      # Topo
        [(-1, -1, 1), (0, 0)], [(-1, -1, -1), (1, 0)], [(1, -1, -1), (1, 1)], [(1, -1, 1), (0, 1)],  # Base
        [(1, -1, -1), (0, 0)], [(1, 1, -1), (1, 0)], [(-1, 1, -1), (1, 1)], [(-1, -1, -1), (0, 1)],  # Esquerda
        [(-1, -1, 1), (0, 0)], [(1, -1, 1), (1, 0)], [(1, 1, 1), (1, 1)], [(-1, 1, 1), (0, 1)],      # Direita
    ]
    # Desenha cada face do cubo aplicando a coordenada de textura e o vértice correspondente.
    for vertex, tex_coord in vertices:
        glTexCoord2f(*tex_coord)  # Aplica a coordenada de textura para o vértice.
        glVertex3fv(vertex)  # Desenha o vértice no espaço 3D.
    glEnd()  # Finaliza o desenho dos quadrados (faces).

In [6]:
# Função para desenhar um paralelepípedo texturizado (para o carro)
def draw_textured_parallelepiped(texture):
    glBindTexture(GL_TEXTURE_2D, texture)  # Associa a textura fornecida ao objeto 3D (paralelepípedo)
    glBegin(GL_QUADS)  # Inicia a definição de quadrados (faces do paralelepípedo)
    # Define os vértices e coordenadas de textura para cada face do paralelepípedo
    vertices = [
        [(1, -0.5, -1.5), (0, 0)], [(1, 0.5, -1.5), (1, 0)], [(1, 0.5, 1.5), (1, 1)], [(1, -0.5, 1.5), (0, 1)],       # Frente
        [(-1, -0.5, 1.5), (0, 0)], [(-1, 0.5, 1.5), (1, 0)], [(-1, 0.5, -1.5), (1, 1)], [(-1, -0.5, -1.5), (0, 1)],   # Trás
        [(-1, 0.5, -1.5), (0, 0)], [(-1, 0.5, 1.5), (1, 0)], [(1, 0.5, 1.5), (1, 1)], [(1, 0.5, -1.5), (0, 1)],       # Topo
        [(-1, -0.5, 1.5), (0, 0)], [(-1, -0.5, -1.5), (1, 0)], [(1, -0.5, -1.5), (1, 1)], [(1, -0.5, 1.5), (0, 1)],   # Base
        [(1, -0.5, -1.5), (0, 0)], [(1, 0.5, -1.5), (1, 0)], [(-1, 0.5, -1.5), (1, 1)], [(-1, -0.5, -1.5), (0, 1)],   # Esquerda
        [(-1, -0.5, 1.5), (0, 0)], [(1, -0.5, 1.5), (1, 0)], [(1, 0.5, 1.5), (1, 1)], [(-1, 0.5, 1.5), (0, 1)],       # Direita
    ]
    # Desenha cada face do paralelepípedo aplicando a coordenada de textura e o vértice correspondente
    for vertex, tex_coord in vertices:
        glTexCoord2f(*tex_coord)  # Aplica a coordenada de textura para o vértice
        glVertex3fv(vertex)  # Desenha o vértice no espaço 3D
    glEnd()  # Finaliza o desenho dos quadrados (faces)

In [7]:

# Função para desenhar o chão texturizado
def draw_ground(texture):
    glBindTexture(GL_TEXTURE_2D, texture)  # Associa a textura fornecida ao chão.
    glBegin(GL_QUADS)  # Inicia a definição de quadrados (o chão será um quadrado com textura).
    # Define os vértices do chão e aplica as coordenadas de textura para cada vértice.
    glTexCoord2f(0, 0)  # Coordenada de textura para o primeiro vértice
    glVertex3fv((-10, -1, -10))  # Vértice no espaço 3D
    glTexCoord2f(1, 0)  # Coordenada de textura para o segundo vértice
    glVertex3fv((10, -1, -10))  # Vértice no espaço 3D
    glTexCoord2f(1, 1)  # Coordenada de textura para o terceiro vértice
    glVertex3fv((10, -1, 10))  # Vértice no espaço 3D
    glTexCoord2f(0, 1)  # Coordenada de textura para o quarto vértice
    glVertex3fv((-10, -1, 10))  # Vértice no espaço 3D
    glEnd()  # Finaliza o desenho do quadrado (chão).


In [8]:
# Função para desenhar uma árvore texturizada
def draw_tree(trunk_texture, leaf_texture):
    glPushMatrix()  # Salva o estado atual da matriz de transformação para o tronco.
    glBindTexture(GL_TEXTURE_2D, trunk_texture)  # Associa a textura do tronco.
    glColor3fv((1, 1, 1))  # Define a cor do tronco (branca, mas a textura será aplicada sobre isso).
    glRotatef(-90, 1, 0, 0)  # Rotaciona o tronco para ficar na vertical.
    quadric = gluNewQuadric()  # Cria um objeto quadrático (usado para desenhar cilindros e esferas).
    gluQuadricTexture(quadric, GL_TRUE)  # Ativa o uso de texturas para o objeto quadrático.
    gluCylinder(quadric, 0.3, 0.3, 2.0, 32, 32)  # Desenha o tronco como um cilindro (raio = 0.3, altura = 2.0).
    glPopMatrix()  # Restaura o estado da matriz de transformação após desenhar o tronco.
    glPushMatrix()  # Salva o estado da matriz novamente para a copa da árvore.
    glBindTexture(GL_TEXTURE_2D, leaf_texture)  # Associa a textura das folhas.
    glColor3fv((1, 1, 1))  # Define a cor das folhas (branca, mas a textura será aplicada sobre isso).
    glTranslatef(0, 2.0, 0)  # Posiciona a copa da árvore no topo do tronco (2.0 unidades acima).
    gluSphere(quadric, 1.0, 32, 32)  # Desenha a copa da árvore como uma esfera (raio = 1.0).
    glPopMatrix()  # Restaura o estado da matriz após desenhar a copa.

In [9]:
# Função para desenhar uma piramide texturizada (para o prédio)
def draw_pyramid(texture):
    glBindTexture(GL_TEXTURE_2D, texture)  # Associa a textura à pirâmide
    glBegin(GL_TRIANGLES)  # Inicia o desenho das faces triangulares

    # Base da pirâmide (quadrado)
    glTexCoord2f(0, 0); glVertex3fv((-1, -1, -1))
    glTexCoord2f(1, 0); glVertex3fv((1, -1, -1))
    glTexCoord2f(1, 1); glVertex3fv((1, -1, 1))

    glTexCoord2f(1, 1); glVertex3fv((1, -1, 1))
    glTexCoord2f(0, 1); glVertex3fv((-1, -1, 1))
    glTexCoord2f(0, 0); glVertex3fv((-1, -1, -1))

    # Lados da pirâmide (triângulos)
    # Lado 1
    glTexCoord2f(0.5, 1); glVertex3fv((0, 1, 0))
    glTexCoord2f(0, 0); glVertex3fv((-1, -1, -1))
    glTexCoord2f(1, 0); glVertex3fv((1, -1, -1))

    # Lado 2
    glTexCoord2f(0.5, 1); glVertex3fv((0, 1, 0))
    glTexCoord2f(0, 0); glVertex3fv((1, -1, -1))
    glTexCoord2f(1, 0); glVertex3fv((1, -1, 1))

    # Lado 3
    glTexCoord2f(0.5, 1); glVertex3fv((0, 1, 0))
    glTexCoord2f(0, 0); glVertex3fv((1, -1, 1))
    glTexCoord2f(1, 0); glVertex3fv((-1, -1, 1))

    # Lado 4
    glTexCoord2f(0.5, 1); glVertex3fv((0, 1, 0))
    glTexCoord2f(0, 0); glVertex3fv((-1, -1, 1))
    glTexCoord2f(1, 0); glVertex3fv((-1, -1, -1))

    glEnd()  # Finaliza o desenho

In [10]:
# Função para desenhar uma esfera texturizada (para o sol)
def draw_sun(texture):
    glPushMatrix()  # Salva o estado atual da matriz de transformação.
    glTranslatef(-7, 12, 0)  # Move o sol para a posição desejada no topo da cena.
    glBindTexture(GL_TEXTURE_2D, texture)  # Associa a textura do sol.
    gluSphere(gluNewQuadric(), 2.0, 32, 32)  # Desenha uma esfera com raio de 2.0 para o sol.
    glPopMatrix()  # Restaura o estado da matriz após desenhar o sol.

In [11]:
# Função para controlar a câmera
def control_camera():
    keys = pygame.key.get_pressed()  # Obtém o estado atual de todas as teclas pressionadas
    if keys[K_LEFT]:
        glRotatef(1, 0, 1, 0)  # Rotaciona a cena ou câmera 1 grau no sentido anti-horário ao redor do eixo Y
    if keys[K_RIGHT]:
        glRotatef(-1, 0, 1, 0)  # Rotaciona a cena ou câmera 1 grau no sentido horário ao redor do eixo Y
    if keys[K_UP]:
        glTranslatef(0, 0, 0.3)    # Move a câmera ou a cena 0.3 unidades para frente no eixo Z
    if keys[K_DOWN]:
        glTranslatef(0, 0, -0.3)  # Move a câmera ou a cena 0.3 unidades para trás no eixo Z

In [12]:
# Função principal
def main():
    init_window()   # Inicializa a janela com OpenGL e Pygame
    init_lighting() # Inicializa a iluminação da cena

    # Carrega as texturas para diferentes objetos
    ground_texture = load_texture("chao.jpg")
    building_texture = load_texture("predio.jpg")
    car_texture = load_texture("carro.jpg")
    car2_texture = load_texture("carro2.jpg")
    trunk_texture = load_texture("tronco.jpg")
    leaf_texture = load_texture("folha.jpg")
    pyramid_texture = load_texture("piramide.jpg")
    sun_texture = load_texture("sol.jpg")

    # Define as posições iniciais dos carros
    car_position = -5  
    car2_position = 5  

    glEnable(GL_TEXTURE_2D)  # Habilita o uso de texturas na cena

    # Loop principal da aplicação
    while True:
        for event in pygame.event.get():  # Verifica os eventos de saída
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()

        control_camera()  # Controla a posição da câmera com as teclas de seta
       
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)  # Limpa os buffers para desenhar a cena

        # Desenha o chão, o sol, os prédios, a pirâmide, as árvores e os carros
        draw_ground(ground_texture)
        draw_sun(sun_texture)

        # Desenha os prédios
        for x in range(-7, -2, 2):
            glPushMatrix()
            glTranslatef(x, 2, -8)
            glScalef(1, 3, 1)  
            draw_textured_cube(building_texture)
            glPopMatrix()

        # Desenha a pirâmide
        glPushMatrix()
        glTranslatef(5, 3, -8)
        glScalef(4, 4, 1)
        draw_pyramid(pyramid_texture)
        glPopMatrix()

        # Desenha as árvores
        for x in range(-7, 8, 2):
            glPushMatrix()
            glTranslatef(x, -1, 8)
            draw_tree(trunk_texture, leaf_texture)
            glPopMatrix()

        # Desenha o carro 1
        glPushMatrix()
        glTranslatef(car_position, -0.5, -2.5)
        draw_textured_parallelepiped(car_texture)
        glPopMatrix()

        # Desenha o carro 2
        glPushMatrix()
        glTranslatef(car2_position, -0.5, 2.5)
        draw_textured_parallelepiped(car2_texture)
        glPopMatrix()

        # Movimento do carro 1 e 2
        car_position += 0.1  # Move o carro 1
        if car_position > 10:
            car_position = -10  # Resetando a posição quando ultrapassa o limite

        car2_position -= 0.1  # Move o carro 2 no sentido contrário
        if car2_position < -10:
            car2_position = 10  # Resetando a posição quando ultrapassa o limite

        pygame.display.flip()  # Atualiza a tela
        pygame.time.wait(10)   # Pausa a execução por 10 milissegundos

# Inicia o programa
if __name__ == "__main__":
    main()

error: video system not initialized